# Notebook
This is just a way of making it easier to go through the audio features, and share any work that I've been doing. This is done by Nicolas

This will probably be a bit messy, but it exists and will be commented and that's what matters

In [1]:
import pandas as pd
import numpy as np
from model_trainer_and_tester import *

# train_set = read_in_dataset([], 'train')
#
# test_set = read_in_dataset([], 'test')

In [2]:
from model.load_data import train_test_split, train_test_split_LSTM

datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "../results_merged_fixedf0/"

batch_size = 64
n_timesteps = 100
train_ratio = 0.4 # ratio of meetings the model is trained on
LSTM_units = 20

#X_train_df, Y_train_df, X_test_df, Y_test_df= train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

In [3]:
features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
shift = [-1, 0, 1]

X_train, y_train = read_in_dataset_lstm(features, shift, to_read='train')

D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_y.iloc[-1] = 1.0
D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_y.iloc[-1] = 1.0
D:\Programming\University\social-signal-processing-project\source\model_trainer_and_tester.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [6]:
y_train.shape

(46640, 3, 1)